In [14]:
import sys
sys.path.append("..") 
from utils.surface_gases import *

In [15]:
COUPLER_options = {}

COUPLER_options['Phi_global'] = 1.0e-9
COUPLER_options['T_outgas'] = 3000.0

COUPLER_options['radius'] = 6.371e6
COUPLER_options['mass']   = 5.972e24
COUPLER_options['gravity'] = 6.67e-11 * COUPLER_options['mass'] / (COUPLER_options['radius']**2)
COUPLER_options['hydrogen_earth_oceans'] = 5.0
COUPLER_options['CH_ratio'] = 2.0
COUPLER_options['fO2_shift_IW'] = 5.0
COUPLER_options['nitrogen_ppmw'] = 2.0
COUPLER_options['planet_coresize'] = 0.55

solvepp_vols = ['H2O', 'CO2', 'N2', 'H2', 'CO', 'CH4']
for v in solvepp_vols:
    COUPLER_options[v+"_included"] = 0
    COUPLER_options[v+"_pa"] = 0.0

COUPLER_options["H2O_included"] = 1
COUPLER_options["CO2_included"] = 1
COUPLER_options["N2_included"]  = 1

COUPLER_options["H2O_pa"] = 1.0e5



In [16]:
COUPLER_options["mantle_mass"] = calc_mantle_mass(COUPLER_options)

In [17]:
solvepp_target = solvepp_get_target_from_params(COUPLER_options)
# solvepp_target = solvepp_get_target_from_pressures(COUPLER_options)

print("Targets:")
print(solvepp_target)

Targets:
{'H': 7.749999999999999e+20, 'C': 1.5499999999999997e+21, 'N': 8.061672162064889e+18}


In [18]:
p_d = solvepp_equilibrium_atmosphere(solvepp_target, COUPLER_options)

p_bar = {}
for e in p_d.keys():
    p_bar[e] = p_d[e]*1.0e-5 # convert to bar

mubar = 0.0
ptot = 0.0
print("Partial pressures")
for v in solvepp_vols:
    print("    %s : %.1f bar"%(v, p_bar[v]))
    ptot += p_bar[v]
    mubar += p_bar[v]*molar_mass[v]
mubar /= ptot

print(mubar)

m_d = {}
for v in solvepp_vols:
    m_d[v] = p_bar[v]*1.0e5 * 4 * 3.14 * (molar_mass[v]/mubar) * COUPLER_options["radius"]**2.0 / COUPLER_options["gravity"]

print(solvepp_atmosphere_mass(p_bar, COUPLER_options))

print(" ")
print(m_d)


C_kg = (m_d["CO2"]/molar_mass["CO2"] + m_d["CO"]/molar_mass["CO"] + m_d["CH4"]/molar_mass["CH4"]) * molar_mass['C']
H_kg = (2*m_d["H2O"]/molar_mass["H2O"] + 2*m_d["H2"]/molar_mass["H2"] + 4*m_d["CH4"]/molar_mass["CH4"]) *molar_mass['H']
N_kg = m_d["N2"]

print(H_kg, C_kg, N_kg)


Partial pressures
    H2O : 1815.9 bar
    CO2 : 609.6 bar
    N2 : 1.4 bar
    H2 : 0.0 bar
    CO : 0.0 bar
    CH4 : 0.0 bar
0.02455004188782538
{'H2O': 6.925929032754683e+21, 'CO2': 5.679418864982086e+21, 'N2': 8.061672161840598e+18, 'H': 7.749999899279672e+20, 'C': 1.5499999997114256e+21, 'N': 8.061672161840598e+18}
 
{'H2O': 6.922417881898108e+21, 'CO2': 5.676539641657918e+21, 'N2': 8.057585237619655e+18, 'H2': 0.0, 'CO': 0.0, 'CH4': 0.0}
7.746532082713444e+20 1.5492142157680812e+21 8.057585237619655e+18


In [19]:
H_mol = H_kg / (1.0e-3)
H_oc  = H_mol / (7.69e22 * 2.0)


print(H_oc)
print("C/H = %.4f" % (C_kg/H_kg))

5.036756880827986
C/H = 1.9999
